In [1]:
import numpy as np
import os
import sys
import tarfile

In [2]:
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
%matplotlib inline

## Variables to set

Make sure to have cloned this repo for the object detection and research libraries:

https://github.com/tensorflow/models

In [3]:
path_to_frozen = "/home/tyler/git_projects/CarND-Capstone/train_light_detector/carla_inference_graph.pb/frozen_inference_graph.pb"

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)
# min probability to call a traffic light
min_prob = 0.50
# path to images
PATH_TO_TEST_IMAGES_DIR = '../imgs/just_traffic_light/'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, image_name) for
                                  image_name in os.listdir(PATH_TO_TEST_IMAGES_DIR)]
# number of images to view when testing below
n_images_show = 3

In [4]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(path_to_frozen, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [5]:
category_index = {1: 'red', 2: 'yellow', 3: 'green', 4: 'none'}

In [6]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [8]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for image_path in TEST_IMAGE_PATHS[:n_images_show]:
            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            max_score_index = np.argmax(scores[0])
            max_score = scores[0][max_score_index]
            if max_score > min_prob:
                predicted_light = category_index[classes[0][max_score_index]]
            else:
                predicted_light = category_index[4]
            print("Predicted Light: {}".format(predicted_light))

Predicted Light: yellow
Predicted Light: none
Predicted Light: red
